In [1]:
search_for = 129
start_from = 0
threads = 10
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.06659150123596191
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 423524321
type: [1, 1, 1, 1, 129] 129
cases of this type: 276922881
100000 14.125221857486693
200000 25.685488503456913
300000 29.00561917420706
400000 30.784016957945987
500000 31.643025193808807
600000 32.67983277365819
700000 30.510328256644794
800000 31.01977302278258
900000 30.693927610601442
1000000 30.97807010694226
1100000 13.28326562072703
1200000 27.13997359957367
1300000 31.21445512697324
1400000 30.760515541650314
1500000 30.56643732219078
1600000 31.612345890869832
1700000 31.64610144780799
1800000 31.161745736338244
1900000 30.627945486038936
2000000 31.377641689179516
2100000 31.547183280286497
2200000 8.10309025747426
2300000 35.67789566427039
2400000 38.057211304384865
2500000 37.722521114419244
2600000 37.24650543098662
2700000 36.19479290628278
2800000 36.161256123316555
2900000 35.52252306347806
3000000 35.589

29500000 28.093228715264885
29600000 30.56484163441766
29700000 30.482671015607416
29800000 27.042126239172205
29900000 16.250980657204835
30000000 28.63794525651107
30100000 31.868126484896983
30200000 30.626737427735886
30300000 20.460042589344724
30400000 23.695996604676306
30500000 28.698037485028035
30600000 30.799890524005686
30700000 30.75602479349439
30800000 28.153352591625573
30900000 29.68017164244372
31000000 26.387737565971022
31100000 25.641978324570232
31200000 13.409713662635973
31300000 26.208754977030484
31400000 29.030566561474622
31500000 26.979074967093794
31600000 29.046121658156967
31700000 28.841948423192058
31800000 28.234512143584215
31900000 30.3129137867638
32000000 12.425398367465768
32100000 30.228698580664695
32200000 30.31609856122754
32300000 29.15827698218126
32400000 31.033830142288107
32500000 11.109139353085595
32600000 29.86929456579849
32700000 28.810135601816665
32800000 28.02958754275725
32900000 30.116797590784785
33000000 26.97458952224554
331

59200000 24.76335321131837
59300000 23.68564048470704
59400000 23.50699516696824
59500000 24.98386547357524
59600000 25.5260050327562
59700000 11.5215351385402
59800000 27.12908620644673
59900000 27.527710502241117
60000000 28.456018609031936
60100000 27.176449252810762
60200000 28.377630404075518
60300000 28.392411238182387
60400000 28.1480666036823
60500000 28.300711457792236
60600000 16.008716351800857
60700000 23.18384858759263
60800000 26.23671470103243
60900000 25.914181545968205
61000000 25.92060804592755
61100000 25.074609875338123
61200000 13.081419054928428
61300000 22.497670841797657
61400000 25.731221490671537
61500000 25.62342321420232
61600000 25.15325635014435
61700000 26.833652108036613
61800000 15.608115460632213
61900000 24.16628161731837
62000000 28.8100424464183
62100000 28.541375052263945
62200000 30.148723482106522
62300000 28.034327444508047
62400000 28.80504413559204
62500000 27.4141064408539
62600000 28.026782470548284
62700000 27.859566452071864
62800000 10.17

88900000 22.35384326398069
89000000 23.070660963857733
89100000 12.602565336177635
89200000 18.89920776744354
89300000 22.91098864055045
89400000 22.674844791286606
89500000 12.244280066211855
89600000 21.858431212899614
89700000 22.91693505195742
89800000 24.46559421004194
89900000 23.567192330480133
90000000 26.686801117357792
90100000 25.059782683654543
90200000 24.955517390829037
90300000 25.23789420071429
90400000 24.63515948823592
90500000 26.768372076618977
90600000 24.18576882762643
90700000 22.92271024080171
90800000 23.05224797566801
90900000 10.926629618122021
91000000 22.41693914113346
91100000 23.603125585610325
91200000 22.303180084994192
91300000 9.136713033560012
91400000 21.98105656937082
91500000 23.887161815509888
91600000 15.607241458849476
91700000 16.686426887471413
91800000 23.03070225549015
91900000 22.178710300163022
92000000 24.7696823487077
92100000 24.424803055210024
92200000 24.613106938652944
92300000 24.909144491731734
92400000 22.963432481233024
92500000

118000000 21.52571304238295
118100000 23.816249171160884
118200000 20.933714947263116
118300000 21.937518388718367
118400000 20.0450156754945
118500000 19.602479547584476
118600000 20.967072394652103
118700000 19.290050254308994
118800000 20.17624199948435
118900000 19.590704303648955
119000000 12.484065437341922
119100000 17.218213521972054
119200000 7.308234332200519
119300000 14.055438019907042
119400000 16.25724930730242
119500000 19.343548406200078
119600000 21.112395852573613
119700000 19.744353822930403
119800000 20.151715182357144
119900000 20.116149173383683
120000000 19.712059897416513
120100000 22.174540741050347
120200000 21.43549064827698
120300000 22.849326118224944
120400000 21.693044118328263
120500000 19.899113329321807
120600000 21.040264466817543
120700000 19.803468295044485
120800000 20.392976965215944
120900000 20.612756557531384
121000000 19.81918595032887
121100000 20.701738278722978
121200000 8.13838653260933
121300000 13.631242246101369
121400000 14.38005493792

...

290100000 5.303482249392886
290200000 4.533430612370579
290300000 5.189001658195099
290400000 4.9619400414604575
290500000 5.166749532630312
290600000 5.052201297830737
290700000 4.665145067344996
290800000 3.5639046456104535
290900000 1.1180700364088276
291000000 1.1121929267887884
291100000 5.783914460257331
291200000 4.6618911860727215
291300000 5.171481327061515
291400000 5.050988355322907
291500000 4.921364966889101
291600000 4.85897702445328
291700000 4.7504434922923995
291800000 5.167763422128823
291900000 4.936910149380786
292000000 6.084194353546615
292100000 5.3081431244235295
292200000 5.185614983962261
292300000 5.093457113979682
292400000 4.646151729296202
292500000 5.145827983501692
292600000 4.872828064365955
292700000 5.135066825930784
292800000 4.704121651977387
292900000 5.165715027107893
293000000 1.097188998933377
293100000 1.0952462321371854
293200000 3.8994400336031827
293300000 4.665946267045015
293400000 4.91194928935475
293500000 5.102040858608125
293600000 4.8

319200000 3.8556328058957803
319300000 4.253477808615899
319400000 4.203084265778917
319500000 4.0000328295168055
319600000 3.9806221871369964
319700000 4.2305649620679375
319800000 4.323400358346977
319900000 5.376017492749532
320000000 5.4311819538840895
320100000 4.681880778121387
320200000 4.50144041788711
320300000 4.162397338046515
320400000 4.617099288160448
320500000 4.67867494929247
320600000 4.677954624668871
320700000 4.064474526873715
320800000 5.325046750067382
320900000 1.2779766338775296
321000000 0.8600438115049496
321100000 2.771927802659798
321200000 5.026175899928725
321300000 4.155721423126166
321400000 4.716806965557881
321500000 4.647337822995478
321600000 4.59741358118937
321700000 4.319629976361996
321800000 4.5550540194243485
321900000 4.714833837271339
322000000 5.515271265317445
322100000 6.379821880623549
322200000 5.335256497059312
322300000 5.356855064363453
322400000 5.043888448560053
322500000 5.131314694071623
322600000 5.288714835249229
322700000 5.324

348000000 1.9081567523501501
348100000 1.5515965192574044
348200000 1.3131729074277816
348300000 1.2513724255766914
348400000 1.2769872598355902
348500000 1.3592563663828732
348600000 2.091699657523534
348700000 1.6562750048180626
348800000 0.5478825670796414
348900000 0.35064903802002606
349000000 0.8511789632838276
349100000 1.439192023596555
349200000 1.332028969477733
349300000 1.361293526534868
349400000 1.3743664985595623
349500000 1.2149815669300397
349600000 1.4346909227377302
349700000 1.493310969194456
349800000 1.3942200738984294
349900000 1.4765072811935291
350000000 1.5689710270025075
350100000 1.6603166631212771
350200000 1.514959896393823
350300000 1.299311562292721
350400000 1.327286655611276
350500000 1.3235069324359205
350600000 1.2975370640731514
350700000 1.3129150807709702
350800000 1.2689829188650483
350900000 1.0435770334055006
351000000 0.4779824029067543
351100000 0.3205539672327558
351200000 1.9181761074756782
351300000 1.6422551044484357
351400000 1.417329627

376000000 0.1680528849677245
376100000 0.16790737913604975
376200000 0.16734991845492256
376300000 0.16704111265278457
376400000 0.16668936043267246
376500000 0.16655507127777605
376600000 0.16655618254809312
376700000 0.1660019315322002
376800000 0.1653346877415564
376900000 0.16531993922999172
377000000 0.1818580135359049
377100000 0.17111587699896627
377200000 0.169880470623019
377300000 0.16984140342594983
377400000 0.16811488072202935
377500000 0.16067762575727965
377600000 0.16248231227156124
377700000 0.184292288962645
377800000 0.1612136985550344
best so far: 0
type: [1, 1, 3, 1, 43] 129
cases of this type: 30769209
377900000 1.0256049913141192
378000000 0.9960203541466236
378100000 1.181221647216173
378200000 1.5428290525357446
378300000 1.4734901813770427
378400000 1.0016151778319902
378500000 1.1212435909221232
378600000 1.2060078949932787
378700000 1.1542075315250337
378800000 1.3730108023475112
378900000 1.0552646134556805
379000000 1.0993265490415929
379100000 0.888586839

404000000 0.8199037957964836
404100000 0.6596267214702487
404200000 0.791372697528953
404300000 0.8314511921698112
404400000 0.7921365723175169
404500000 0.7572645153414209
404600000 0.6731847889564215
404700000 0.9513567894743866
404800000 0.6882144685855157
404900000 0.7396987169881284
405000000 0.7823355178172331
405100000 0.890042641274745
405200000 0.7328320826581181
405300000 0.6515858762954831
405400000 0.8295776372117162
405500000 0.7692981568757752
405600000 0.6837813987632341
405700000 0.7854595375626711
405800000 0.7548396880926371
405900000 0.8118511929199258
406000000 0.6397815999389966
406100000 0.7511041295934678
406200000 0.7292509580183414
406300000 0.7335739453299045
406400000 0.7235637487038236
406500000 0.7236684187534154
406600000 0.6638956562669946
406700000 0.681671153676188
406800000 0.6557643530413402
406900000 0.6993783259835177
407000000 0.6291938038743417
407100000 0.7197970838310759
407200000 0.7831514144801398
407300000 0.685738171822083
407400000 0.676788